# Combined Compression Example

This notebook demonstrates how to use both mesh encoding and array compression features together.

We'll cover:

1. Creating a mesh with vertex attributes (positions, normals, colors)
2. Compressing the mesh and additional arrays
3. Saving everything to a single zip file
4. Loading and reconstructing the data

In [1]:
import os
import numpy as np
import zipfile
import json

from meshoptimizer import (
    Mesh,
    encode_mesh,
    decode_mesh,
    encode_array,
    decode_array,
    save_encoded_mesh_to_zip,
    load_encoded_mesh_from_zip,
    save_array_to_zip,
    load_array_from_zip,
    save_arrays_to_zip,
    load_arrays_from_zip
)

## Creating a Mesh with Attributes

Let's create a simple mesh (a cube) with vertex positions, normals, and colors:

In [2]:
# Create vertex positions (a cube)
positions = np.array([
    # positions          
    [-0.5, -0.5, -0.5],
    [0.5, -0.5, -0.5],
    [0.5, 0.5, -0.5],
    [-0.5, 0.5, -0.5],
    [-0.5, -0.5, 0.5],
    [0.5, -0.5, 0.5],
    [0.5, 0.5, 0.5],
    [-0.5, 0.5, 0.5]
], dtype=np.float32)

# Create indices
indices = np.array([
    0, 1, 2, 2, 3, 0,  # front
    1, 5, 6, 6, 2, 1,  # right
    5, 4, 7, 7, 6, 5,  # back
    4, 0, 3, 3, 7, 4,  # left
    3, 2, 6, 6, 7, 3,  # top
    4, 5, 1, 1, 0, 4   # bottom
], dtype=np.uint32)

# Create normals (simplified - just using position as normal)
normals = np.array([
    [-1, -1, -1],
    [1, -1, -1],
    [1, 1, -1],
    [-1, 1, -1],
    [-1, -1, 1],
    [1, -1, 1],
    [1, 1, 1],
    [-1, 1, 1]
], dtype=np.float32)
# Normalize the normals
normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

# Create colors (one for each vertex)
colors = np.array([
    [1, 0, 0, 1],  # red
    [0, 1, 0, 1],  # green
    [0, 0, 1, 1],  # blue
    [1, 1, 0, 1],  # yellow
    [1, 0, 1, 1],  # magenta
    [0, 1, 1, 1],  # cyan
    [1, 1, 1, 1],  # white
    [0.5, 0.5, 0.5, 1]  # gray
], dtype=np.float32)

# Create a Mesh object with positions and indices
mesh = Mesh(positions, indices)
print(f"Created mesh with {len(positions)} vertices and {len(indices)} indices")
print(f"Normals shape: {normals.shape}")
print(f"Colors shape: {colors.shape}")

Created mesh with 8 vertices and 36 indices
Normals shape: (8, 3)
Colors shape: (8, 4)


## Additional Data

Let's also create some additional data that might be associated with the mesh:

In [3]:
# Create texture coordinates
texcoords = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 1],
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 1]
], dtype=np.float32)

# Create some animation data (e.g., bone weights)
bone_weights = np.random.random((8, 4)).astype(np.float32)
# Normalize weights to sum to 1
bone_weights = bone_weights / bone_weights.sum(axis=1, keepdims=True)

# Create some metadata
metadata = {
    "name": "Cube",
    "material": "Default",
    "tags": ["example", "cube", "simple"]
}

print(f"Texture coordinates shape: {texcoords.shape}")
print(f"Bone weights shape: {bone_weights.shape}")

Texture coordinates shape: (8, 2)
Bone weights shape: (8, 4)


## Compressing and Saving Everything

Now let's compress all the data and save it to a single zip file:

In [4]:
# Encode the mesh (positions and indices)
encoded_mesh = encode_mesh(positions, indices)
print(f"Encoded mesh: vertex buffer size = {len(encoded_mesh.vertices)} bytes, "
      f"index buffer size = {len(encoded_mesh.indices)} bytes")

# Encode the additional arrays
encoded_normals = encode_array(normals)
encoded_colors = encode_array(colors)
encoded_texcoords = encode_array(texcoords)
encoded_bone_weights = encode_array(bone_weights)

print(f"Encoded normals size: {len(encoded_normals.data)} bytes")
print(f"Encoded colors size: {len(encoded_colors.data)} bytes")
print(f"Encoded texcoords size: {len(encoded_texcoords.data)} bytes")
print(f"Encoded bone weights size: {len(encoded_bone_weights.data)} bytes")

Encoded mesh: vertex buffer size = 65 bytes, index buffer size = 37 bytes
Encoded normals size: 56 bytes
Encoded colors size: 83 bytes
Encoded texcoords size: 67 bytes
Encoded bone weights size: 143 bytes


Now let's save everything to a single zip file:

In [5]:
# Create a zip file
zip_path = "combined_data.zip"

# Create a dictionary of encoded arrays
encoded_arrays = {
    "normals": encoded_normals,
    "colors": encoded_colors,
    "texcoords": encoded_texcoords,
    "bone_weights": encoded_bone_weights
}

# Use the new utility function to save everything to a zip file
from meshoptimizer.ziputils import save_combined_data_to_zip

try:
    save_combined_data_to_zip(
        encoded_mesh=encoded_mesh,
        encoded_arrays=encoded_arrays,
        metadata=metadata,
        zip_path=zip_path
    )
    print(f"Saved all data to {zip_path}")
    print(f"Zip file size: {os.path.getsize(zip_path)} bytes")
    
    # Calculate total size of all original data
    total_size = (
        positions.nbytes + 
        indices.nbytes + 
        normals.nbytes + 
        colors.nbytes + 
        texcoords.nbytes + 
        bone_weights.nbytes
    )
    print(f"Total original data size: {total_size} bytes")
    print(f"Compression ratio: {os.path.getsize(zip_path) / total_size:.2f}")
except Exception as e:
    print(f"Error saving data: {str(e)}")

Saved all data to combined_data.zip
Zip file size: 1542 bytes
Total original data size: 656 bytes
Compression ratio: 2.35


## Loading and Reconstructing the Data

Now let's load the data from the zip file and reconstruct everything:

In [6]:
# Load data from the zip file using the new utility function
from meshoptimizer.ziputils import load_combined_data_from_zip

try:
    # Load all data at once
    loaded_encoded_mesh, loaded_encoded_arrays, loaded_metadata = load_combined_data_from_zip(zip_path)
    print("Loaded all data from zip file")
except Exception as e:
    print(f"Error loading data: {str(e)}")

Loaded all data from zip file


Now let's reconstruct the mesh:

In [7]:
# Decode the mesh
loaded_positions, loaded_indices = decode_mesh(loaded_encoded_mesh)
print(f"Decoded mesh with {len(loaded_positions)} vertices and {len(loaded_indices)} indices")

Decoded mesh with 8 vertices and 36 indices


Now let's reconstruct the arrays:

In [8]:
# Decode the arrays
loaded_normals = decode_array(loaded_encoded_arrays["normals"])
loaded_colors = decode_array(loaded_encoded_arrays["colors"])
loaded_texcoords = decode_array(loaded_encoded_arrays["texcoords"])
loaded_bone_weights = decode_array(loaded_encoded_arrays["bone_weights"])

print(f"Decoded normals shape: {loaded_normals.shape}")
print(f"Decoded colors shape: {loaded_colors.shape}")
print(f"Decoded texcoords shape: {loaded_texcoords.shape}")
print(f"Decoded bone weights shape: {loaded_bone_weights.shape}")

Decoded normals shape: (8, 3)
Decoded colors shape: (8, 4)
Decoded texcoords shape: (8, 2)
Decoded bone weights shape: (8, 4)


## Verifying the Data

Let's verify that the loaded data matches the original data:

In [9]:
# Verify mesh data
positions_match = np.allclose(loaded_positions, positions, rtol=1e-5)
indices_match = np.array_equal(loaded_indices, indices)
print(f"Positions match: {positions_match}")
print(f"Indices match: {indices_match}")

# Verify array data
normals_match = np.allclose(loaded_normals, normals, rtol=1e-5)
colors_match = np.allclose(loaded_colors, colors, rtol=1e-5)
texcoords_match = np.allclose(loaded_texcoords, texcoords, rtol=1e-5)
bone_weights_match = np.allclose(loaded_bone_weights, bone_weights, rtol=1e-5)
print(f"Normals match: {normals_match}")
print(f"Colors match: {colors_match}")
print(f"Texture coordinates match: {texcoords_match}")
print(f"Bone weights match: {bone_weights_match}")

# Verify metadata
metadata_match = loaded_metadata == metadata
print(f"Metadata match: {metadata_match}")

Positions match: True
Indices match: False
Normals match: True
Colors match: True
Texture coordinates match: True
Bone weights match: True
Metadata match: True


## Cleaning Up

Let's clean up the files we created:

In [ ]:
# Clean up
if os.path.exists(zip_path):
    os.remove(zip_path)
    print(f"Removed {zip_path}")

print("\nExample completed successfully!")